```mermaid
flowchart TB
A[广州数据]
B[上海数据]
C[深圳数据]
D[广州查询工具QueryEngineTool]
E[上海查询工具QueryEngineTool]
F[深圳查询工具QueryEngineTool]
G[文档查询选择工具get_tools]
H[生成prompt工具create_system_prompt]
I[创建代理工具create_agent]
J[最终代理builder_agent]

A--->D
B--->E
C--->F
D--->G
E--->G
F--->G
H--->J
G--->J
I--->J
```

### 1. 把QueryEngine转为工具使用，并使用tool_retriever.retrieve选择工具

在这个脚本中，首先构建了针对不同城市（广州、深圳、上海）的查询引擎，这些引擎被封装为工具（`QueryEngineTool`），并存储在 `tool_dict` 字典中。通过定义一个检索器 `tool_retriever` 并调用 `tool_retriever.retrieve(task)` 方法来选择与当前任务最相关的工具。这样可以确保在面对不同查询时，能够动态地从现有的工具集中选取合适的查询引擎进行信息检索。

### 2. 首先创建prompt，然后选择QueryEngine工具创建Agent

为了构建一个能够解决特定任务的代理（agent），首先需要生成一个系统提示（system prompt）。这一步通过调用 `create_system_prompt` 函数来完成。随后使用这个系统提示以及从 `tool_retriever.retrieve(task)` 选取的相关工具，利用 `ReActAgent.from_tools` 方法创建最终的代理。这种方法能够确保在每次面对新的查询任务时都能够动态生成和调整系统提示及工具集。

### 3. 嵌套Agent的使用

脚本中展示了如何构建一个多级代理（嵌套agent）。首先通过一系列步骤创建了一个顶层代理（builder_agent），它负责生成和管理其他具体的代理。这种设计允许在高层实现逻辑控制，同时利用具体执行层（如 `QueryEngineTool`）来处理细节任务。这样的结构可以提高系统的灵活性和可扩展性。

### 4. 嵌套的Agent带来流程不稳定，输出不如直接RAG靠谱

虽然多级代理（嵌套agent）提供了一种灵活的方式来构建复杂的任务解决系统，但在实际应用中可能会遇到一些挑战。例如，在某些情况下，顶层代理可能不能准确地判断出最适合当前任务的具体工具集或策略，这可能导致生成的系统提示不够精确或者查询结果偏离预期目标。此外，这种结构也可能增加代码复杂性和维护难度。

相比之下，直接使用RAG（Retrieval-Augmented Generation）方法在处理信息检索和回答问题时更为直接有效，因为它可以直接基于现有文档进行上下文依赖的信息提取与融合生成，通常能提供更稳定且高质量的输出结果。

In [11]:
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

base_url='http://localhost:11434'
llm = Ollama(model="qwen2.5:latest", request_timeout=360.0,base_url=base_url)
Settings.llm = llm
Settings.embed_model = OllamaEmbedding(model_name="quentinz/bge-large-zh-v1.5:latest",base_url=base_url)

## 获取数据

In [12]:
# 利用https://baike.deno.dev/获取百度百科查询地址
import re
import requests
from bs4 import BeautifulSoup

def get_html(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    
    # 检查响应状态码
    if response.status_code == 200:
        return response
    else:
        print(f"请求失败，状态码：{response.status_code}")
        return None

def build_md_from_html(html,save_path):
    soup = BeautifulSoup(html, 'html.parser')

    md_content=''
    # 获取summary部分
    md_content+='# 概览\n'
    summary_div= soup.find_all('div', class_='J-summary')  # 替换为实际的数据选择器
    summary_paragraphs = summary_div[0].find_all('div', {'data-tag': 'paragraph'})
    for paragraph in summary_paragraphs:
        text = paragraph.get_text(strip=True)
        text=re.sub(r'\[[1-9][0-9]*\]','',text)
        md_content+=text+'\n\n'
    
    # 获取内容部分
    content_div = soup.find_all('div', class_='J-lemma-content')  # 替换为实际的数据选择器
    paragraphs = content_div[0].find_all('div', {'data-tag': ['paragraph','header']})
    for paragraph in paragraphs:
        data_tag=paragraph['data-tag']
        text = paragraph.get_text(strip=True)

        if data_tag=='header':
            text=text.replace('播报编辑','')
            data_level=int(paragraph['data-level'])
            text='#'*data_level+' '+text+'\n'
        else:
            text=re.sub(r'\[[1-9][0-9]*\]','',text)
            text+='\n\n'
        
        md_content+=text

    # 将结果写入文件
    with open(save_path, 'w', encoding='utf-8') as file:
        file.write(md_content)

cities = ["广州", "深圳", "上海"]
# for city in cities:
#     url = f"https://baike.deno.dev/item/{city}"
#     response = get_html(url).json()
#     baike_url=response['data']['link']
#     print(f"{city} 的链接是：{baike_url}")

#     html=get_html(baike_url).text
#     print(f'get {city} html done')

#     save_path=f'../../data/citys/{city}.md'
#     build_md_from_html(html,save_path)
#     print(f'save {city} markdown')


## 每个文件构建查询引擎

In [13]:
from llama_index.core import SimpleDirectoryReader

city_docs = {}
for city in cities:
    city_docs[city] = SimpleDirectoryReader(
        input_files=[f"../../data/citys/{city}.md"]
    ).load_data()

In [14]:
from llama_index.core import VectorStoreIndex
from llama_index.core.tools import QueryEngineTool,ToolMetadata

tool_dict = {}

for city in cities:
    index=VectorStoreIndex.from_documents(documents=city_docs[city])

    query_engine=index.as_query_engine()

    tool=QueryEngineTool(
        query_engine=query_engine,
        metadata=ToolMetadata(
            name=city,
            description=(f"关于{city}问题的回答助手")))
    tool_dict[city]=tool

## prompt->选择查询工具->创建Agent进行增强生成

In [15]:
from llama_index.core.objects import ObjectIndex

tool_index=ObjectIndex.from_objects(
    list(tool_dict.values()),
    index_cls=VectorStoreIndex
)

tool_retriever=tool_index.as_retriever(similarity_top_k=1)

In [16]:
from llama_index.core.llms import ChatMessage
from llama_index.core import ChatPromptTemplate
from llama_index.core.agent import ReActAgent
from typing import List
from llama_index.core.agent import FunctionCallingAgent

GEN_SYS_PROMPT_STR = """\
Task information is given below. 

Given the task, please generate a system prompt for an  bot to solve this task: 
{task} \
"""

gen_sys_prompt_messages = [
    ChatMessage(
        role="system",
        content="You are helping to build a system prompt for another bot.",
    ),
    ChatMessage(role="user", content=GEN_SYS_PROMPT_STR),
]
GEN_SYS_PROMPT_TMPL = ChatPromptTemplate(gen_sys_prompt_messages)


agent_cache = {}

def create_system_prompt(task: str)-> str:
    """Create system prompt for another agent given an input task."""
    fmt_messages = GEN_SYS_PROMPT_TMPL.format_messages(task=task)
    response = llm.chat(fmt_messages)
    return response.message.content


def get_tools(task: str) -> List[str]:
    """Get the set of relevant tools to use given an input task."""
    subset_tools = tool_retriever.retrieve(task)
    tool_names= [t.metadata.name for t in subset_tools]
    return tool_names


def create_agent(system_prompt: str, tool_names: List[str]) -> str:
    ''' Create an agent given a system prompt and an input set of tools
    system_prompt :  prompt created by create_system_prompt
    tool_names : tools can only choice 广州 上海 or 深圳
    '''
    try:
        # print('tool_names',tool_names)
        # get the list of tools
        input_tools = [tool_dict[tn] for tn in tool_names]
 
        agent = ReActAgent.from_tools(input_tools, verbose=True)
        agent_cache["agent"] = agent
        return_msg = "Agent created successfully."
    except Exception as e:
        return_msg = f"An error occurred when building an agent. Here is the error: {repr(e)}"
    return return_msg

In [17]:
from llama_index.core.tools import FunctionTool

system_prompt_tool = FunctionTool.from_defaults(fn=create_system_prompt)
get_tools_tool = FunctionTool.from_defaults(fn=get_tools)
create_agent_tool = FunctionTool.from_defaults(fn=create_agent)

## 创建汇总Agent

In [18]:
GPT_BUILDER_SYS_STR = """\
You are helping to construct an agent given a user-specified task. 
You should generally use the tools in this order to build the agent.

1) Create system prompt tool: to create the system prompt for the agent.
2) Get tools tool: to fetch the candidate set of tools to use.
3) Create agent tool: to create the final agent.
"""

prefix_msgs = [ChatMessage(role="system", content=GPT_BUILDER_SYS_STR)]


builder_agent = FunctionCallingAgent.from_tools(
    tools=[system_prompt_tool, get_tools_tool, create_agent_tool],
    prefix_messages=prefix_msgs,
    verbose=True
)

In [19]:
response=builder_agent.query("创建一个代理，告诉我关于广州的事情")

> Running step ea222b84-cd51-4cb1-8905-e25b338ee64d. Step input: 创建一个代理，告诉我关于广州的事情
Added user message to memory: 创建一个代理，告诉我关于广州的事情
=== Calling Function ===
Calling function: create_system_prompt with args: {"task": "\u63d0\u4f9b\u5173\u4e8e\u5e7f\u5dde\u7684\u4fe1\u606f"}
=== Function Output ===
以下是为解决此任务生成的系统提示：

```
您需要为某个机器人编写对话脚本，使其能够提供关于广州的相关信息。下面是具体的指示和指南。

【目标】：生成一段对话，内容是关于广州市的基本信息，包括但不限于地理位置、著名景点、文化特色等。

【对话示例框架】：
1. 用户提问或表达兴趣。
2. 机器人基于预设的信息库进行回答。
3. 可以根据用户进一步的询问提供更详细的信息。

【提示要点】：
- 地理位置：位于中国南部，广东省省会城市。
- 著名景点：珠江夜游、白云山、广州塔（小蛮腰）、陈家祠等。
- 文化特色：广府文化、早茶文化、粤菜特色等。
- 其他相关信息：人口数量、面积大小、著名历史事件或人物等。

【示例对话】：
用户：我想了解一些关于广州的信息。
机器人：当然可以！广州位于中国南部的广东省，是省会城市。这里有许多著名的景点和美食。比如珠江夜游非常受欢迎，还有白云山和广州塔（小蛮腰）等。另外，广州还是广府文化的发源地之一，您可以品尝到地道的早茶和粤菜。

用户：广州有什么特别的文化传统吗？
机器人：广州拥有丰富多样的文化特色，例如早茶文化、广东音乐等。特别是早茶文化，在广州非常流行，许多老店提供正宗的粤式点心。
```
> Running step 88119ea0-335b-4d54-a6e1-6efb19fdb1fe. Step input: None
=== Calling Function ===
Calling function: get_tools with args: {"task": "\u63d0\u4f9b\u5173\u4e8e\u5e7f\u5dde\

In [20]:
agent_cache["agent"].query('广州所有的4A级旅游景点')

> Running step b2a18ce0-648a-41ae-8271-f1b92b781d8b. Step input: 广州所有的4A级旅游景点
Thought: 我需要使用 广州 工具来获取广州所有4A级的旅游景点信息。
Action: 广州
Action Input: {'input': '广州 4A级 旅游景点'}
Observation: 广州的4A级旅游景点包括：

1. 中山纪念堂 - 越秀区东风中路259号
2. 莲花山旅游区 - 番禺区石楼镇西门路18号
3. 宝墨园 - 番禺区沙湾镇紫坭村
4. 西汉南越王博物馆 - 广州市解放北路867号
5. 黄花岗公园 - 广州市先烈中路79号
6. 越秀公园 - 广州市解放北路988号
7. 碧水湾温泉度假村 - 从化流溪温泉旅游度假区（良口）
8. 华南植物园 - 广州市天河区天源路1190号
9. 广州动物园 - 广州市先烈中路120号
10. 正佳广场商贸旅游区 - 广州市天河路228号
> Running step 0e0c0f06-7719-40ed-8dc5-0dac9a370137. Step input: None
Thought: 我可以使用获取到的信息来回答用户的问题，我将用中文作答。
Answer: 广州的4A级旅游景点包括：中山纪念堂（地址：越秀区东风中路259号）、莲花山旅游区（地址：番禺区石楼镇西门路18号）、宝墨园（地址：番禺区沙湾镇紫坭村）、西汉南越王博物馆（地址：广州市解放北路867号）、黄花岗公园（地址：广州市先烈中路79号）、越秀公园（地址：广州市解放北路988号）、碧水湾温泉度假村（地址：从化流溪温泉旅游度假区，良口镇）、华南植物园（地址：广州市天河区天源路1190号）、广州动物园（地址：广州市先烈中路120号）和正佳广场商贸旅游区（地址：广州市天河路228号）。


Response(response='广州的4A级旅游景点包括：中山纪念堂（地址：越秀区东风中路259号）、莲花山旅游区（地址：番禺区石楼镇西门路18号）、宝墨园（地址：番禺区沙湾镇紫坭村）、西汉南越王博物馆（地址：广州市解放北路867号）、黄花岗公园（地址：广州市先烈中路79号）、越秀公园（地址：广州市解放北路988号）、碧水湾温泉度假村（地址：从化流溪温泉旅游度假区，良口镇）、华南植物园（地址：广州市天河区天源路1190号）、广州动物园（地址：广州市先烈中路120号）和正佳广场商贸旅游区（地址：广州市天河路228号）。', source_nodes=[NodeWithScore(node=TextNode(id_='fb7ee9fe-07a0-49dc-84e3-02c298396473', embedding=None, metadata={'file_path': '..\\..\\data\\citys\\广州.md', 'file_name': '广州.md', 'file_size': 166691, 'creation_date': '2024-12-26', 'last_modified_date': '2024-12-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='03b5365f-9a5f-4ef7-8ed7-9147ed60f78b', node_type='4', metadata={'file_path': '..\\..\\data\\citys\\广州.md', 'file_name': '广州.md', 'file_

In [23]:
# 直接使用RAG
response=tool_dict['广州'].query_engine.query('广州所有的4A级旅游景点')
print(response.response)

广州的4A级旅游景点包括：

1. 番禺长隆旅游度假区
2. 白云山风景名胜区
3. 莲花山旅游区
4. 宝墨园
5. 西汉南越王博物馆
6. 黄花岗公园
7. 越秀公园
8. 碧水湾温泉度假村
9. 华南植物园
10. 广州动物园
11. 正佳广场商贸旅游区


In [24]:
agent_cache["agent"].query('广州的民间手工艺有哪些？')

> Running step 96a66369-191e-442c-bbb8-a9f92c885a9a. Step input: 广州的民间手工艺有哪些？
Thought: 我需要使用工具来获取关于广州民间手工艺的信息。
Action: 广州
Action Input: {'input': '广州的民间手工艺有哪些？'}
Observation: 广州的民间手工艺包括广绣、广彩、玉雕和木雕。这些传统工艺各具特色：

- 广绣是珠三角地区的刺绣工艺总称，与潮州刺绣合称为粤绣，是中国四大名绣之一。
- 广彩是一种釉上彩瓷，又称广州织金彩瓷，色彩鲜艳，具有浓厚的岭南地方特色和中西合璧的艺术风格。
- 玉雕是以玉石为材料进行雕刻的传统工艺，技艺精湛，作品精美。
- 木雕以酸枝木、紫檀木等木材为主料，制作出各种精美的工艺品。
> Running step 9a3586da-8f52-4a7a-86bb-623cb92b4ee6. Step input: None
Thought: 我可以回答这个问题了。
Answer: 广州的民间手工艺包括广绣、广彩、玉雕和木雕。其中：

1. 广绣是珠三角地区的刺绣工艺总称，与潮州刺绣合称为粤绣，是中国四大名绣之一。
2. 广彩是一种釉上彩瓷，又称广州织金彩瓷，色彩鲜艳，具有浓厚的岭南地方特色和中西合璧的艺术风格。
3. 玉雕是以玉石为材料进行雕刻的传统工艺，技艺精湛，作品精美。
4. 木雕以酸枝木、紫檀木等木材为主料，制作出各种精美的工艺品。


Response(response='广州的民间手工艺包括广绣、广彩、玉雕和木雕。其中：\n\n1. 广绣是珠三角地区的刺绣工艺总称，与潮州刺绣合称为粤绣，是中国四大名绣之一。\n2. 广彩是一种釉上彩瓷，又称广州织金彩瓷，色彩鲜艳，具有浓厚的岭南地方特色和中西合璧的艺术风格。\n3. 玉雕是以玉石为材料进行雕刻的传统工艺，技艺精湛，作品精美。\n4. 木雕以酸枝木、紫檀木等木材为主料，制作出各种精美的工艺品。', source_nodes=[NodeWithScore(node=TextNode(id_='71c6a1af-f1ce-4a0a-88a4-dc5bf5609e3a', embedding=None, metadata={'file_path': '..\\..\\data\\citys\\广州.md', 'file_name': '广州.md', 'file_size': 166691, 'creation_date': '2024-12-26', 'last_modified_date': '2024-12-26'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='8e451c75-002a-45e9-96c9-eb7dcca0c629', node_type='4', metadata={'file_path': '..\\..\\data\\citys\\广州.md', 'file_name': '广州.md', 'file_size': 166691, 'creation_date': '2024-12-26', 'last_modif

In [25]:
# 直接使用RAG
response=tool_dict['广州'].query_engine.query('广州的民间手工艺有哪些？')
print(response.response)

广州的民间手工艺主要包括广绣、广彩、玉雕和木雕等。这些传统工艺各具特色：

- 广绣：作为珠三角民间刺绣工艺的总称，以其构图饱满、形象传神、纹理清晰、色泽富丽以及针法多样而著称，是中国四大名绣之一。
- 广彩：全称为“广州织金彩瓷”，是一种以销往外国为主要目的的瓷器，色彩鲜艳、构图繁密、金碧辉煌，具有浓郁的地方特色和中西合璧的艺术风格。
- 玉雕：加工雕琢玉石的传统工艺，根据不同玉料的特点进行设计与琢磨，制作成精美工艺品。
- 木雕：广州木雕以雕工精细著称，常用酸枝木、紫檀木等材料，制成家具、樟木箱等多种成品。
